In [12]:
import sys
sys.path.append("../")

In [13]:
import torch
import numpy as np
import json
import matplotlib.pyplot as plt
import cv2
import os
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms
from tqdm import tqdm
import pandas as pd

from seger.datasets.seg_dataset import LaneDataset
from seger.models.networks import get_network
from seger.loss import get_loss
from seger.metric import IOU,fscore_batch,fscore
from Notebooks.utils import *
import glob

In [14]:
config_files = glob.glob("../Experiments/*/*.json")

In [15]:
config_files.sort()
print(config_files)

['../Experiments/exp1_lane/baseline-1.json', '../Experiments/exp2_lane/baseline-2.json', '../Experiments/exp3_lane/baseline-3.json', '../Experiments/exp4_lane/baseline-4.json', '../Experiments/exp5_lane/baseline-5.json', '../Experiments/exp6_lane/baseline-6.json', '../Experiments/exp7_lane/baseline-7.json', '../Experiments/exp8_lane/baseline-8.json']


# Single Experiment Loss IOU Calculation

In [16]:
config_file = "../cfgs/baseline-8.json"

In [17]:
with open(config_file) as fp:
    cfg = json.load(fp)
    
arch_name = cfg["arch"]["arch_name"]
experiment_name = cfg["model_stuff"]["exp_name"]
MODEL_PATH = "../Experiments/{}/best.pth".format(experiment_name)
cfg["arch"]["arch_kwargs"]["pretrained"] = False
input_size = cfg["train"]["width"], cfg["train"]["height"]

cfg["data"]["train_data"] = "../" + cfg["data"]["train_data"]
cfg["data"]["valid_data"] = "../" + cfg["data"]["valid_data"]
cfg["data"]["train_root"] = "../" + cfg["data"]["train_root"]
cfg["data"]["label_root"] = "../" + cfg["data"]["label_root"]

train_file = cfg["data"]["train_data"]
val_file = cfg["data"]["valid_data"]
train_root = cfg["data"]["train_root"]
label_root = cfg["data"]["label_root"]

model = get_network(cfg["data"]["input_channels"], cfg["data"]["num_classes"], cfg["arch"]["arch_name"], cfg["arch"]["arch_kwargs"])
model.cuda()
model.load_state_dict(torch.load(MODEL_PATH)["model_weights"])

loss_kwargs = {"num_classes": cfg["data"]["num_classes"], "weights": cfg["train"]["loss_kwargs"]["weights"]}
criterion = get_loss(cfg["train"]["loss"], loss_kwargs)

Initializing SCNN...


In [18]:
train_dataset = LaneDataset(cfg, augmentations = True , train=True)
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=cfg["train"]["batch_size"],
                                                       num_workers=2, shuffle=True)

In [19]:
val_dataset = LaneDataset(cfg, augmentations = False , train=False)
valloader = torch.utils.data.DataLoader(val_dataset, batch_size=cfg["train"]["batch_size"], num_workers=2,
                                                     shuffle=True)

In [20]:
loaders = [trainloader,valloader]

In [21]:
data = [experiment_name]

for loader in loaders:
    
    loss_counter = []
    acc_counter = []
    
    for k in tqdm(loader):
        with torch.set_grad_enabled(False):

            model.zero_grad()
            inputs, targets = k


            if cfg["device"]["use_gpu"]:
                inputs = inputs.cuda()
                targets = targets.cuda()

            outputs = model(inputs)

            if isinstance(outputs, tuple):
                _, __ = outputs
                outputs = _  

            final_loss = criterion(outputs, targets).data.item()
            iou = fscore_batch(targets,outputs)

            loss_counter.append(final_loss)
            acc_counter.append(iou)

    mean_iou = np.mean(iou)
    mean_loss = np.mean(final_loss)

    data.append(mean_loss)
    data.append(mean_iou)
    


100%|██████████| 7/7 [00:02<00:00,  2.45it/s]


In [22]:
data

['exp8_lane',
 0.15743012726306915,
 0.27086318679644,
 0.22220154106616974,
 0.20721008102847605]

# Overall  Analysis

In [23]:
df = pd.DataFrame()

In [24]:
for config_file in config_files:

    with open(config_file) as fp:
        cfg = json.load(fp)

    arch_name = cfg["arch"]["arch_name"]
    optimizer_name = cfg["train"]["optim"]
    lr_scheduler_name = cfg["train"]["sched_name"]
    loss_name = cfg["train"]["loss"]
    loss_weights = str(cfg["train"]["loss_kwargs"]["weights"][0]) + ":" + str(cfg["train"]["loss_kwargs"]["weights"][1])
        
    experiment_name = cfg["model_stuff"]["exp_name"]
    MODEL_PATH = "../Experiments/{}/best.pth".format(experiment_name)
    cfg["arch"]["arch_kwargs"]["pretrained"] = False
    input_size = cfg["train"]["width"], cfg["train"]["height"]

    cfg["data"]["train_data"] = "../" + cfg["data"]["train_data"]
    cfg["data"]["valid_data"] = "../" + cfg["data"]["valid_data"]
    cfg["data"]["train_root"] = "../" + cfg["data"]["train_root"]
    cfg["data"]["label_root"] = "../" + cfg["data"]["label_root"]
    

    train_file = cfg["data"]["train_data"]
    val_file = cfg["data"]["valid_data"]
    train_root = cfg["data"]["train_root"]
    label_root = cfg["data"]["label_root"]

    model = get_network(cfg["data"]["input_channels"], cfg["data"]["num_classes"], cfg["arch"]["arch_name"], cfg["arch"]["arch_kwargs"])
    model.cuda()
    model.load_state_dict(torch.load(MODEL_PATH)["model_weights"])

    loss_kwargs = {"num_classes": cfg["data"]["num_classes"], "weights": cfg["train"]["loss_kwargs"]["weights"]}
    criterion = get_loss(cfg["train"]["loss"], loss_kwargs)


    train_dataset = LaneDataset(cfg, augmentations = True , train=True)
    trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=cfg["train"]["batch_size"],
                                                           num_workers=2, shuffle=True)

    val_dataset = LaneDataset(cfg, augmentations = False , train=False)
    valloader = torch.utils.data.DataLoader(val_dataset, batch_size=cfg["train"]["batch_size"], num_workers=2,
                                                         shuffle=True)

    loaders = [trainloader,valloader]

    data = [experiment_name]

    for loader in loaders:

        loss_counter = []
        acc_counter = []

        for k in tqdm(loader):
            with torch.set_grad_enabled(False):

                model.zero_grad()
                inputs, targets = k


                if cfg["device"]["use_gpu"]:
                    inputs = inputs.cuda()
                    targets = targets.cuda()

                outputs = model(inputs)

                if isinstance(outputs, tuple):
                    _, __ = outputs
                    outputs = _  

                final_loss = criterion(outputs, targets).data.item()
                iou = fscore_batch(targets,outputs)

                loss_counter.append(final_loss)
                acc_counter.append(iou)

        mean_iou = np.mean(iou)
        mean_loss = np.mean(final_loss)

        data.append(mean_loss)
        data.append(mean_iou)
        
    print(data)

    result_dict = {"exp_name":data[0], 
                   "arch_name" : arch_name, 
                   "optimizer_name":optimizer_name,
                   "lr_scheduler_name":lr_scheduler_name,
                   "loss_name":loss_name,
                   "loss_weights":loss_weights,
                   "train_loss":data[1],"train_iou":data[2],
                   "val_loss":data[3],
                   "val_iou":data[4]}
    
    df = df.append(result_dict,ignore_index=True)


Initializing Deeplabv3...


100%|██████████| 2/2 [00:02<00:00,  1.41s/it]


Initializing SCNN...


100%|██████████| 7/7 [00:03<00:00,  2.13it/s]


Initializing SCNN...


...

  0%|          | 0/57 [00:00<?, ?it/s]

Initializing ERFNet...


100%|██████████| 7/7 [00:02<00:00,  2.41it/s]


Initializing SCNN...


100%|██████████| 7/7 [00:03<00:00,  2.11it/s]


In [25]:
df

,arch_name,exp_name,loss_name,loss_weights,lr_scheduler_name,optimizer_name,train_iou,train_loss,val_iou,val_loss
0,deeplabv3,exp1_lane,weighed_ce_loss,1:200,sgdr,adam,0.041956,0.284672,0.034351,0.344927
1,enet,exp2_lane,weighed_ce_loss,1:200,sgdr,sgd,0.032687,0.484465,0.037739,0.309494
2,scnn,exp3_lane,weighed_ce_loss,1:100,sgdr,sgd,0.160970,0.203453,0.076592,0.198199
3,scnn,exp4_lane,dice,1:100,sgdr,sgd,0.006782,0.999798,0.025242,0.999964
4,deeplabv3,exp5_lane,weighed_ce_loss,1:200,StepLR,adam,0.042474,0.350330,0.034330,0.307646
5,erfnet,exp6_lane,weighed_ce_loss,1:100,sgdr,sgd,0.163666,0.118408,0.082704,0.195643
6,erfnet,exp7_lane,weighed_ce_loss,1:25,sgdr,sgd,0.292846,0.093007,0.161362,0.220175
7,scnn,exp8_lane,weighed_ce_loss,1:25,sgdr,sgd,0.292293,0.140799,0.137654,0.224384


In [27]:
df.to_csv("../experiment_result_summary.csv",index=False)